In [ ]:
import os
import urllib.request

In [ ]:
if (not os.path.exists('tinystories_vocab.csv')):
  !git clone https://huggingface.co/datasets/schuler/TinyStories4Pascal
  !unzip TinyStories4Pascal/tinystories_tokenized_81.csv.zip
  !unzip TinyStories4Pascal/tinystories_vocab.csv.zip

In [ ]:
!apt-get update && apt-get -y install fpc fpc-source lazarus git subversion

In [ ]:
!svn checkout https://svn.code.sf.net/p/cai/svncode/trunk/lazarus neural-api

U    neural-api/neural/neuralfit.pas
Checked out revision 2011.


In [ ]:
!svn checkout https://svn.code.sf.net/p/lazarus-ccr/svn/components/multithreadprocs mtprocs

In [ ]:
!lazbuild mtprocs/multithreadprocslaz.lpk

In [ ]:
code = """
program smallTransformer;
(*
Copyright (C) 2024 Joao Paulo Schwarz Schuler

This program is free software; you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation; either version 2 of the License, or
any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License along
with this program; if not, write to the Free Software Foundation, Inc.,
51 Franklin Street, Fifth Floor, Boston, MA 02110-1301 USA.
*)

{$mode objfpc}{$H+}

uses {$IFDEF UNIX} {$IFDEF UseCThreads}
  cthreads, {$ENDIF} {$ENDIF}
  Classes,
  neuralnetwork,
  neuralvolume,
  neuralfit,
  neuralthread,
  neuraldatasets,
  neuralab,
  CustApp,
  Math,
  sysutils;

const
  csContextLen = 80;
  csTrainingFileName = 'tinystories_tokenized_81.csv';
  csVocabFileName = 'tinystories_vocab.csv';
  csMinSampleSize = 3; // Minimum of 1 token.
  csEmbedDim = 128;
  csModelVocabSize = 3000;

type
  TTestFitLoading = class(TCustomApplication)
  protected
    FDataset: array of array of integer;
    FDictionary: TStringListInt;
    FDatasetSize: integer;
    FNN: THistoricalNets;
    NFit: TNeuralDataLoadingFit;
    FSampler: TNNetSamplerBase;
    FMaxPredictCharPos: integer;
    FVocabSize: integer; // Character based vocabulary/dictionary.
    procedure LoadDataset;
    procedure DoRun; override;
  public
    procedure OnAfterEpoch(Sender: TObject);
    procedure OnAfterStep(Sender: TObject);
    procedure GetTrainingPair(Idx: integer; ThreadId: integer; pInput, pOutput: TNNetVolume);
    procedure GetValidationPair(Idx: integer; ThreadId: integer; pInput, pOutput: TNNetVolume);
    procedure GetTestPair(Idx: integer; ThreadId: integer; pInput, pOutput: TNNetVolume);
  end;

  procedure TTestFitLoading.LoadDataset;
  var
    Tokens: array of integer;
  begin
    WriteLn('Loading vocabulary: ', csVocabFileName);
    FDictionary.LoadFromFile(csVocabFileName);
    WriteLn('Indexing vocabulary with ', FDictionary.Count,' words.');
    FDictionary.SaveCurrentPositionAndSort();
    FDictionary.StringToIntegerArray('one day a', Tokens);
    WriteLn('one day a: ',Tokens[0],' ',Tokens[1],' ',Tokens[2]);

    WriteLn('Dic 93: ', FDictionary.IntegerToWord(93));
    WriteLn('Dic 88: ', FDictionary.IntegerToWord(88));
    WriteLn('Dic 40: ', FDictionary.IntegerToWord(40));

    FilterCSVWithNumbersUpToMax(csTrainingFileName, 'temp.csv', csModelVocabSize-1);
    LoadIntegersInCSV('temp.csv', FDataset);

    FVocabSize := FDictionary.Count;
    FDatasetSize := Length(FDataSet);

    WriteLn('Loaded dataset with ', FDatasetSize, ' rows');
  end;

  function CyclicalAdvLRScheduler25b(Epoch: integer): single;
  var
    BaseLearning: single;
    LocalEpoch: integer;
  begin
    BaseLearning := 0.0005;
    LocalEpoch := Epoch mod 25;
    Result := BaseLearning;
    if Epoch < 25 then
    begin
      if LocalEpoch < 7 then
      begin
        Result := BaseLearning * (1 + 0.5 * LocalEpoch);
      end
      else
      begin
        Result := (BaseLearning * 4) * Power(0.90, (LocalEpoch - 7));
      end;
    end
    else
    begin
      Result := 0.0001;
    end;
  end;

  procedure TTestFitLoading.DoRun;
  var
    W: TNNetLayer;
    I: integer;
    Opt: TNeuralOptimizerAdam;
  begin
    FDictionary := TStringListInt.Create();
    LoadDataset();
    FNN := THistoricalNets.Create();
    Opt := TNeuralOptimizerAdam.Create(0.9, 0.98);
    NFit := TNeuralDataLoadingFit.Create();
    FMaxPredictCharPos := csContextLen;
    FSampler := TNNetSamplerTopP.Create(0.4);
    FNN.AddLayer([
      TNNetInput.Create(csContextLen, 1, 1),
      TNNetTokenAndPositionalEmbedding.Create(csModelVocabSize, csEmbedDim, 0, 0.02, 0.01),
      TNNetPointwiseConvLinear.Create(512)
    ]);

    for I := 1 to 2 do FNN.AddTransformerBlockCAI(8, 2048, true, false, false);

    FNN.AddLayer([
      TNNetPointwiseConvLinear.Create(csEmbedDim),
      TNNetPointwiseConvLinear.Create(csModelVocabSize),
      TNNetPointwiseSoftMax.Create(1)
    ]);

    DebugThreadCount();
    FNN.DebugStructure;
    FNN.DebugWeights();

    WriteLn('Computing...');
    NFit.LogEveryBatches := 100;
    NFit.CustomLearningRateScheduleFn:=@CyclicalAdvLRScheduler25b;
    NFit.Optimizer := Opt;
    NFit.LearningRateDecay := 0.0;
    NFit.StaircaseEpochs := 1;
    NFit.L2Decay := 0.1;
    NFit.EnableMultiClassLoss();
    NFit.EnableClassComparisonInLastPixel();
    NFit.AvgWeightEpochCount := 1;
    NFit.OnAfterEpoch := @OnAfterEpoch;
    NFit.OnAfterStep := @OnAfterStep;
    NFit.FitLoading(
      FNN,
      {TrainingVolumesCount=}48000*1,
      {ValidationVolumesCount=}48000*1 div 20,
      {TestVolumesCount=}48000*1 div 20,
      {batchsize=}32,
      {epochs=}500,
      @GetTrainingPair, @GetValidationPair, @GetTestPair
    );
    FNN.DebugWeights();
    OnAfterEpoch(Self);
    FSampler.Free;
    Opt.Free;
    NFit.Free;
    FNN.Free;
    FDictionary.Free;
    Terminate;
  end;

  procedure TTestFitLoading.OnAfterEpoch(Sender: TObject);
  begin
    WriteLn(GenerateStringFromCasualNN(NFit.NN, FDictionary, 'one day', nil),'.');
    WriteLn(GenerateStringFromCasualNN(NFit.NN, FDictionary, 'once upon a', nil),'.');
    WriteLn(GenerateStringFromCasualNN(NFit.NN, FDictionary, 'once upon a time', nil),'.');
    WriteLn(GenerateStringFromCasualNN(NFit.NN, FDictionary, 'once upon', nil),'.');
    WriteLn(GenerateStringFromCasualNN(NFit.NN, FDictionary, 'billy', FSampler),'.');
  end;

  procedure TTestFitLoading.OnAfterStep(Sender: TObject);
  begin
    //if Random(100)=0 then OnAfterEpoch(Sender);
    //NFit.ThreadNN[0].DebugWeights();
  end;

  procedure TTestFitLoading.GetTrainingPair(Idx: integer; ThreadId: integer;
    pInput, pOutput: TNNetVolume);
  var
    SampleId: integer;
    SampleLen: integer;
    SampleCutPosition: integer;
    ExpectedTokenInt: integer;
    AIntegerArray: array of integer;
  begin
    // Make sure that expected input and output have the proper sizes.
    if FNN.GetFirstLayer().Output.Size <> pInput.Size then pInput.ReSize(FNN.GetFirstLayer().Output);
    if FNN.GetLastLayer().Output.Size <> pOutput.Size then pOutput.ReSize(FNN.GetLastLayer().Output);
    // Get the input sample
    SampleId := Random(FDatasetSize);//Random(Min(48000,FDatasetSize));
    SampleLen := Min(Length(FDataset[SampleId]), pInput.SizeX);
    SampleLen := Min(FMaxPredictCharPos, SampleLen);
    SampleCutPosition := SampleLen;
    // The expected token is the next character in the string
    ExpectedTokenInt := FDataset[SampleId][SampleCutPosition-1];
    // Encode the input volume
    AIntegerArray := Copy(FDataset[SampleId], 0, SampleCutPosition);
    pInput.Fill(0);
    pInput.CopyNoChecksIntArr( AIntegerArray );
    // Encode the output volume (includes the predicted word)
    AIntegerArray := Copy(FDataset[SampleId], 1, SampleCutPosition);
    pOutput.OneHotEncoding(AIntegerArray) ;
    pOutput.Tag := ExpectedTokenInt;
  end;

  procedure TTestFitLoading.GetValidationPair(Idx: integer; ThreadId: integer;
    pInput, pOutput: TNNetVolume);
  var
    SampleId: integer;
    SampleLen: integer;
    SampleCutPosition: integer;
    ExpectedTokenChar: char;
    ExpectedTokenInt: integer;
    AIntegerArray: array of integer;
  begin
    // Make sure that expected input and output have the proper sizes.
    if FNN.GetFirstLayer().Output.Size <> pInput.Size then pInput.ReSize(FNN.GetFirstLayer().Output);
    if FNN.GetLastLayer().Output.Size <> pOutput.Size then pOutput.ReSize(FNN.GetLastLayer().Output);
    // Get the input sample
    SampleId := (Idx * 20) mod FDatasetSize; // Min(FDatasetSize,48000);
    SampleLen := Min(Length(FDataset[SampleId]), pInput.SizeX);
    SampleCutPosition := SampleLen;
    // The expected token is the next character in the string
    ExpectedTokenInt := FDataset[SampleId][SampleCutPosition-1];
    // Encode the input and output volumes
    AIntegerArray := Copy(FDataset[SampleId], 0, SampleCutPosition);
    pInput.Fill(0);
    pInput.CopyNoChecksIntArr( AIntegerArray );
    // Encode the output volume (includes the predicted word)
    AIntegerArray := Copy(FDataset[SampleId], 1, SampleCutPosition);
    pOutput.OneHotEncoding(AIntegerArray) ;
    pOutput.Tag := ExpectedTokenInt;
  end;

  procedure TTestFitLoading.GetTestPair(Idx: integer; ThreadId: integer;
    pInput, pOutput: TNNetVolume);
  begin
    GetValidationPair(Idx, ThreadId, pInput, pOutput);
  end;

var
  Application: TTestFitLoading;
begin
  Application := TTestFitLoading.Create(nil);
  Application.Title:='Nano Covolutional Based NLP Trained from File';
  Application.Run;
  Application.Free;
end.
"""
with open("neural-api/examples/CaiOptimizedDenseNet/CaiOptimizedDenseNet.lpr", "w") as text_file:
    text_file.write(code)
!lazbuild neural-api/examples/CaiOptimizedDenseNet/CaiOptimizedDenseNet.lpi
!ls -l neural-api/bin/x86_64-linux/bin/CaiOptimizedDenseNet

In [ ]:
!neural-api/bin/x86_64-linux/bin/CaiOptimizedDenseNet

Loading vocabulary: tinystories_vocab.csv
Indexing vocabulary with 21438 words.
one day a: 93 88 40
Dic 93: one
Dic 88: day
Dic 40: a
Loaded dataset with 945153 rows
CPU threads reported by the operating system: 8.
 Layers: 148
 Neurons:12857
 Weights:7190528 Weight Sum:  -36.579983 Bias sum:    0.000000 Inertia sum:    0.000000 Delta sum:    0.000000
Has AVX: TRUE Has AVX2: FALSE Has AVX512: FALSE
Layer  0 Neurons:   0 Weights:     0 TNNetInput(80,1,1,0,0) Output:80,1,1 Learning Rate:0.0100 Inertia:0.90 Weight Sum:  0.0000 Bias Sum:  0.0000 Branches:1
Layer  1 Neurons:   1 Weights:384000 TNNetTokenAndPositionalEmbedding(3000,128,0,10000,0) Output:80,1,128 Learning Rate:0.0100 Inertia:0.90 Weight Sum:  8.7327 Bias Sum:  0.0000 Parent:0 Branches:1
Layer  2 Neurons: 512 Weights: 65536 TNNetPointwiseConvLinear(512,1,0,1,0) Output:80,1,512 Learning Rate:0.0100 Inertia:0.90 Weight Sum: -9.6615 Bias Sum:  0.0000 Parent:1 Branches:4
Layer  3 Neurons: 512 Weights:262144 TNNetPointwiseConvLinea

In [ ]:
!cat autosave.csv